In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns 

import scispacy
import spacy
import en_core_sci_sm   #需要先 pip install <Model URL> 对应的包的超链接
from spacy import displacy
from scispacy.abbreviation import AbbreviationDetector
from scispacy.umls_linking import UmlsEntityLinker

1. 我们就用spacy做词语检索和画图
2. 我们用LDA进行主题关键词生成
3. 我们用spacy里的word2vec进行词语匹配，然后再检索(有一个similarity)
4. 我们用每个里面的词语的word2vec制造的词向量，然后用一个模型进行训练，预测其死亡还是没死亡(5w多个患者)

5. bert embedding的参考 https://github.com/kexinhuang12345/clinicalBERT/blob/master/modeling_readmission.py
    
6.https://github.com/cchantra/nlp_tourism  全面的
    
    https://explosion.ai/blog/spacy-transformers
    

    

# 本例中参考的链接:
#https://github.com/kavgan/nlp-in-practice/blob/master/tf-idf/Keyword%20Extraction%20with%20TF-IDF%20and%20SKlearn.ipynb

详解tf-IDF: https://drive.google.com/file/d/1EhucFINkDNEDCsrmyxHr_UWQcdYRmI4e/view

tf-ide这类是count-based不是深度学习类的

In [2]:
df_note=pd.read_csv('NOTEEVENTS.csv')


F:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
import string
import time


def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

    
df_note.TEXT=df_note.TEXT.apply(remove_punct)

In [4]:
#df_note['len_count']=df_note.TEXT.apply(lambda x: len(x))

import re

def pre_process(text):
    
    
    #remove tags
    text=re.sub("</?.*?>"," <> ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text




df_note['TEXT'] = df_note['TEXT'].apply(lambda x:pre_process(x))

#show the first 'text'


df_note['len_count']=df_note.TEXT.apply(lambda x: len(x))
df_note=df_note[df_note['len_count']>50]




In [5]:
#就用这个ipynb作为指导
#https://github.com/kavgan/nlp-in-practice/blob/master/tf-idf/Keyword%20Extraction%20with%20TF-IDF%20and%20SKlearn.ipynb

In [8]:
print('after we clean the data, we have ', df_note.shape)

after we clean the data, we have  (2072810, 12)


In [9]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
  
stop_words = set(stopwords.words('english')) 
  

def get_stop_word(text, stop_list): #clear useless stop words
    result=[]
    text=word_tokenize(text)
    for i in text:
        if i not in stop_list and len(i)>2:
            result.append(i)
    return ' '.join(result)


begin=time.time()
df_note['TEXT']=df_note['TEXT'].apply(lambda x: get_stop_word(x,stop_words))
print('stop word cost: ',time.time()-begin,' s')

stop word cost:  1773.0621089935303  s


In [10]:
pd.DataFrame.to_csv(df_note,'cleaned_note.csv') #存储清洗后的

In [11]:
df_clean=pd.read_csv('cleaned_note.csv',index_col=0)

F:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
F:\anaconda\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [12]:
df_clean

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT,len_count
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date Discharge Date Service ADDENDUM...,660
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date Discharge Date Date Birth Sex S...,11083
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date Discharge Date Service CARDIOTH...,7970
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date Discharge Date Service MEDICINE...,14405
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date Discharge Date Date Birth Sex S...,12930
...,...,...,...,...,...,...,...,...,...,...,...,...
2083175,2070657,31097,115637.0,2132-01-21,2132-01-21 03:27:00,2132-01-21 03:38:00,Nursing/other,Report,17581.0,NaN,NPN Infant remains sats mild ICRSCR clear equa...,742
2083176,2070658,31097,115637.0,2132-01-21,2132-01-21 09:50:00,2132-01-21 09:53:00,Nursing/other,Report,19211.0,NaN,Neonatology DOL CGA weeks CVR Continues sats o...,513
2083177,2070659,31097,115637.0,2132-01-21,2132-01-21 16:42:00,2132-01-21 16:44:00,Nursing/other,Report,20104.0,NaN,Family Meeting Note Family meeting held Name D...,446
2083178,2070660,31097,115637.0,2132-01-21,2132-01-21 18:05:00,2132-01-21 18:16:00,Nursing/other,Report,16023.0,NaN,NPN Resp Known lastname remains sat spells des...,1688


In [19]:
#【调参可以从这里调】

from nltk.corpus import stopwords #废话word，比如I ，are you
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer


stop_words = set(stopwords.words('english')) 

begin=time.time()
cv2=CountVectorizer(max_df=0.95,stop_words=stop_words,max_features=1000) #调参
docs=df_clean['TEXT'][0:1750000].tolist()
cv_result=cv2.fit_transform(docs)

print('vectorization : ',time.time()-begin,' s')

vectorization :  237.86075806617737  s


In [20]:

from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(cv_result)


def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [21]:
feature_names=cv2.get_feature_names()
len(feature_names)

500

In [26]:
feature_names=cv2.get_feature_names()

# get the document that we want to extract keywords from

#测试一个
doc=df_clean.TEXT[2050000]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv2.transform([doc]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,20) #打印这个doc的主题


for k in keywords:
    print(k,keywords[k])

female 0.62
normal 0.316
hrs 0.238
due 0.192
soft 0.167
sepsis 0.13
start 0.126
aware 0.125
diet 0.124
service 0.124
patent 0.123
management 0.122
evaluation 0.121
tachycardia 0.121
check 0.12
infection 0.12
sat 0.117
location 0.116
pulses 0.115
fever 0.114
